# Before going to main content
new feature was being created.

V1 V319-V320

V7 V109-V110

V12 V329-V330

V14 V316-V331

V18 V4-V5
 

**Sorry, I want to update quickly this kernel, but I can't because these days kernel's commiting is too busy! **

This kernel is using official kernel for judging whether my new feature is meaningful.

The official kernel is here(https://www.kaggle.com/inversion/ieee-simple-xgboost)

And that the feature which I made is explained in detail here ( https://www.kaggle.com/yasagure/how-do-we-treat-with-similar-columns-v319-v321/edit/run/18988983)

# Introduction - Do they match?
**Some people throw away similar columns, but it is good thing?**

Everyone found that there is a lot of similar columns in this dataset.

V319-V320 and V109-V110 are good example of them.

How should we "use" it?

Most people throw away the data, but I think that I can get useful information from it.

**My idea is paying attention to whether they match each other.**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))


In [ ]:
from sklearn import preprocessing
import matplotlib.pylab as plt
%matplotlib inline
import xgboost as xgb

# load dataset

In [ ]:
train_transaction = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('../input/ieee-fraud-detection/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('../input/ieee-fraud-detection/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv', index_col='TransactionID')

We want to both of data -"identity" and "transaction"-

The identity data should be added to transaction data. 

In [ ]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

# How V-feature is?

In [ ]:
train.columns[54:393]

V features have a lot of similar columns.

Let's look at it.

In [ ]:
train.iloc[:,54:393].corr()

# Feature engineering
I picked up these 3 pairs of columns. 

In [ ]:
train.loc[:,["V319","V320"]].corr()

In [ ]:
train.loc[:,["V109","V110"]].corr()

In [ ]:
train.loc[:,["V329","V330"]].corr()

In [ ]:
train.loc[:,["V316","V331"]].corr()

In [ ]:
train.loc[:,["V4","V5"]].corr()

They seem to be similar.

In [ ]:
train["diff_V319_V320"] = np.zeros(train.shape[0])

train.loc[train["V319"]!=train["V320"],"diff_V319_V320"] = 1

test["diff_V319_V320"] = np.zeros(test.shape[0])

test.loc[test["V319"]!=test["V320"],"diff_V319_V320"] = 1

train["diff_V109_V110"] = np.zeros(train.shape[0])

train.loc[train["V109"]!=train["V110"],"diff_V109_V110"] = 1

test["diff_V109_V110"] = np.zeros(test.shape[0])

test.loc[test["V109"]!=test["V110"],"diff_V109_V110"] = 1

train["diff_V329_V330"] = np.zeros(train.shape[0])

train.loc[train["V329"]!=train["V330"],"diff_V329_V330"] = 1

test["diff_V329_V330"] = np.zeros(test.shape[0])

test.loc[test["V329"]!=test["V330"],"diff_V329_V330"] = 1


train["diff_V316_V331"] = np.zeros(train.shape[0])

train.loc[train["V331"]!=train["V316"],"diff_V316_V331"] = 1

test["diff_V316_V331"] = np.zeros(test.shape[0])

test.loc[test["V316"]!=test["V331"],"diff_V316_V331"] = 1


train["diff_V4_V5"] = np.zeros(train.shape[0])

train.loc[train["V4"]!=train["V5"],"diff_V4_V5"] = 1

test["diff_V4_V5"] = np.zeros(test.shape[0])

test.loc[test["V4"]!=test["V5"],"diff_V4_V5"] = 1

# look at appearance of this new feature 

## V319-V320

In [ ]:
plt.bar(train.groupby("diff_V319_V320").mean().isFraud.index,train.groupby("diff_V319_V320").mean().isFraud.values)


## V109-V110

In [ ]:
plt.bar(train.groupby("diff_V109_V110").mean().isFraud.index,train.groupby("diff_V109_V110").mean().isFraud.values)

## V329-V330

In [ ]:
plt.bar(train.groupby("diff_V329_V330").mean().isFraud.index,train.groupby("diff_V329_V330").mean().isFraud.values)

## V316-V331

In [ ]:
plt.bar(train.groupby("diff_V316_V331").mean().isFraud.index,train.groupby("diff_V316_V331").mean().isFraud.values)

## V4-V5

In [ ]:
plt.bar(train.groupby("diff_V4_V5").mean().isFraud.index,train.groupby("diff_V4_V5").mean().isFraud.values)

There seem to be difference, but the gap of "diff_V109_V110" is small.

In Version7, I found that "diff_V109_V110" is not meaningful. I deleted.

In Version13, I found that "diff_V329_V330" is not meaningful. I deleted.

In Version20, I found that "diff_V4_V5" is not meaningful. I deleted.

I feel that when the gap is big, the column is meaningful.

In [ ]:
train = train.drop("diff_V109_V110",axis=1)
test = test.drop("diff_V109_V110",axis=1)

train = train.drop("diff_V329_V330",axis=1)
test = test.drop("diff_V329_V330",axis=1)

train = train.drop("diff_V316_V331",axis=1)
test = test.drop("diff_V316_V331",axis=1)


train = train.drop("diff_V4_V5",axis=1)
test = test.drop("diff_V4_V5",axis=1)


# data cleaning

In [ ]:
print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [ ]:
del train, test, train_transaction, train_identity, test_transaction, test_identity


## Label Encoding
We cannot use literal features for XGB, so these features are changes.

For example, [H,G,W,A] →[0,1,2,3]

The number of the words is often related to the numeral([0,1,2,3]).

In [ ]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

# Model

In [ ]:
clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=4,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        missing=-999)

clf.fit(X_train, y_train)

In [ ]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('simple_xgboost.csv')

# Is it useful?
The score of V5 is 0.9367.

The official score was 0.9366.

The score seem to be  improved.

The score of V8(V109-V110 added) is not unknown. Soon, I tell you it. ←the diff V109-V110 does not seem to be useful.

Also, the diff V329-V330 does not seem to be useful.

# Conclusion
Some people think this is useful, but others not.

If you interested in it, please use for your model and judge wheter these columns are useful for your model.